In [1]:
import os
import json
import re
import string
import random
import time
import datetime

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from argparse import Namespace
from tqdm.notebook import tqdm

# from datasets import Dataset

import transformers
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import pipeline
from transformers import BertTokenizer, DataCollatorForLanguageModeling

import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

In [ ]:
args = Namespace(
    data_path = './processed_data/combined_ir.csv',
    start_idx = 15000,
    end_idx=25000,
    model_path='./models/parallel_three_multi',
    max_samples=2000
)

## Pre-process Data 

In [ ]:
df = pd.read_csv(args.data_path)[0:args.max_samples]

In [12]:
tokenizer = BertTokenizer.from_pretrained('casehold/legalbert')

In [13]:
encoded_data = [tokenizer.encode_plus(row['query'], row['result'], add_special_tokens=True, max_length=512, pad_to_max_length=True, truncation=True) for index,row in df.iterrows()]
input_ids = [item['input_ids'] for item in encoded_data]
attention_masks = [item['attention_mask'] for item in encoded_data]
labels = [row['label'] for index,row in df.iterrows()]

# Convert to tensors
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
labels = torch.tensor(labels)


dataset_test = TensorDataset(input_ids, attention_masks, labels)

dataloader_test = DataLoader(dataset_test, batch_size=16, shuffle=True) # NOTE : maybe set pin_memory=True

/home/jz75/Documents/2023-2024/SH-Project/CS4099-LegalNLP/pytorchenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty e

In [14]:
from transformers import BertTokenizer, BertModel, BertConfig
def evaluate_sequence_pair_class(model_path,  title):
    '''
    Routine for evaluating model for sequence pair classification
    '''
    batch_progress = tqdm(total=len(dataloader_test), desc='Batches', leave=True)

    # load model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    # config = BertConfig.from_pretrained('./models/mlm_model')
    # config.num_labels = 2
    # model = BertForSequenceClassification(config=config)
    # bert_state_dict = torch.load(model_path)
    # model.bert.load_state_dict(bert_state_dict)

    device = 'cpu'

    # Check if cuda available
    if torch.cuda.is_available():
        # model.to('cuda')
        device = 'cuda'

    print(device)

    model.to(device)

    model.eval()

    predictions, true_labels = [], []

    print('Evaluating ' + f'[{title}]')
    print('============================================')


    with torch.no_grad(): # disable calculating gradients (more efficient for evaluation)
        for batch in dataloader_test:
            input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).flatten() # find index of max value in logits tensor (where each index corresponds to a binary class)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            batch_progress.update(1)


    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, acc = precision_recall_fscore_support(true_labels, predictions, average='binary')
    # roc_auc = roc_auc_score(true_labels, predictions)  # Uncomment if ROC-AUC is needed
    print(acc)
    print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}')



In [16]:
evaluate_sequence_pair_class('./models/mlm_model_manual1',  'Sequence Pair Classificaiton Evaluation Metrics')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./models/mlm_model_manual1 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda
Evaluating [Sequence Pair Classificaiton Evaluation Metrics]
1688
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
